<a href="https://colab.research.google.com/github/ahzaidy/Programs/blob/main/CPSC_5440_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install python==3.7

ERROR: Could not find a version that satisfies the requirement python==3.7 (from versions: none)
ERROR: No matching distribution found for python==3.7
